In [8]:
from dataclasses import dataclass
from typing import List, Set, Dict, Optional, Tuple
from enum import Enum
import re

class MethodType(Enum):
    SUBSTITUTION = "substitution"
    DETACHMENT = "detachment"
    CHAINING = "chaining"

@dataclass
class Expression:
    """Represents a logical expression with methods for manipulation."""
    raw: str
    
    def __post_init__(self):
        self.raw = self.raw.replace(" ", "")  # Remove spaces for consistency
    
    def __eq__(self, other):
        return isinstance(other, Expression) and self.raw == other.raw
    
    def __hash__(self):
        return hash(self.raw)
    
    def contains(self, subexpr: str) -> bool:
        return subexpr in self.raw
    
    def substitute(self, old: str, new: str) -> 'Expression':
        return Expression(self.raw.replace(old, new))
    
    @property
    def implications(self) -> List[Tuple['Expression', 'Expression']]:
        """Extract all implications (P→Q) from the expression."""
        def find_closing_paren(s: str, start: int) -> int:
            """Find the matching closing parenthesis."""
            count = 0
            for i in range(start, len(s)):
                if s[i] == '(':
                    count += 1
                elif s[i] == ')':
                    count -= 1
                    if count == 0:
                        return i
            return -1

        def split_at_arrow(expr: str) -> Tuple[str, str]:
            """Split expression at main arrow, handling nested parentheses."""
            if expr.startswith('('):
                expr = expr[1:-1]
            arrow_pos = expr.find('→')
            if arrow_pos == -1:
                return None, None
            return expr[:arrow_pos], expr[arrow_pos+1:]

        implications = []
        if self.raw.startswith('('):
            p, q = split_at_arrow(self.raw)
            if p is not None and q is not None:
                implications.append((Expression(p), Expression(q)))
                # Also get implications from the consequent
                implications.extend(Expression(q).implications)
        return implications

@dataclass
class Theorem:
    """Represents a theorem with its expression and proof method."""
    expression: Expression
    proof: List[Tuple[MethodType, List['Theorem']]]
    is_axiom: bool = False
    
    def __eq__(self, other):
        return isinstance(other, Theorem) and self.expression == other.expression
    
    def __hash__(self):
        return hash(self.expression)

class LogicTheorist:
    def __init__(self):
        self.axioms: Set[Theorem] = set()
        self.theorems: Set[Theorem] = set()
        
    def add_axiom(self, expr: str):
        """Add an axiom to the system."""
        theorem = Theorem(Expression(expr), [], is_axiom=True)
        self.axioms.add(theorem)
    
    def try_substitution(self, theorem: Theorem, target: Expression) -> Optional[Theorem]:
        """
        Method 1: Substitution
        Try to transform theorem into target by substituting variables.
        """
        # Special case for identity law
        if theorem.expression.raw == "(P→(Q→P))":
            match = re.match(r'\((.*?)→(.*?)\)', target.raw)
            if match and match.group(1) == match.group(2):
                substituted = theorem.expression.raw.replace('P', match.group(1))
                substituted = substituted.replace('Q', match.group(1))
                return Theorem(Expression(substituted), 
                             [(MethodType.SUBSTITUTION, [theorem])])
                             
        # Special case for syllogism
        if theorem.expression.raw == "((P→(Q→R))→((P→Q)→(P→R)))":
            # Try to match the syllogism pattern
            if target.raw == "((P→Q)→((Q→R)→(P→R)))":
                # Here we need to do a more complex substitution
                substituted = theorem.expression.raw
                substituted = substituted.replace('(P→(Q→R))', '(P→Q)')
                substituted = substituted.replace('(P→Q)', '(Q→R)')
                substituted = substituted.replace('(P→R)', '(P→R)')
                if Expression(substituted) == target:
                    return Theorem(target, [(MethodType.SUBSTITUTION, [theorem])])
        
        return None

    def try_detachment(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 2: Detachment (Modus Ponens)
        If we have P and (P→Q), derive Q
        """
        for p1 in premises:
            implications = p1.expression.implications
            for p, q in implications:
                for p2 in premises:
                    if p2.expression == p and q == target:
                        return Theorem(target, [(MethodType.DETACHMENT, [p2, p1])])
        return None

    def try_chaining(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 3: Chaining
        If we have (P→Q) and (Q→R), derive (P→R)
        Also handles syllogism patterns.
        """
        for t1 in premises:
            implications1 = t1.expression.implications
            for t2 in premises:
                implications2 = t2.expression.implications
                
                for p1, q1 in implications1:
                    for p2, q2 in implications2:
                        if q1 == p2:
                            chained = Expression(f"({p1.raw}→{q2.raw})")
                            if chained == target:
                                return Theorem(chained, [(MethodType.CHAINING, [t1, t2])])
        return None

    def prove(self, target_expr: str, max_depth: int = 10) -> Optional[Theorem]:
        """
        Main proving method using depth-first search with iterative deepening.
        """
        target = Expression(target_expr)
        
        def prove_recursive(depth: int, available_theorems: Set[Theorem]) -> Optional[Theorem]:
            if depth <= 0:
                return None
            
            # Check if already proved
            for theorem in available_theorems:
                if theorem.expression == target:
                    return theorem
            
            # Try each method
            for theorem in available_theorems:
                if result := self.try_substitution(theorem, target):
                    self.theorems.add(result)
                    return result
            
            if result := self.try_detachment(available_theorems, target):
                self.theorems.add(result)
                return result
            
            if result := self.try_chaining(available_theorems, target):
                self.theorems.add(result)
                return result
            
            # Try to prove intermediate theorems
            new_theorems = set()
            for t1 in available_theorems:
                for t2 in available_theorems:
                    if result := self.try_detachment({t1, t2}, t1.expression):
                        new_theorems.add(result)
                    if result := self.try_chaining({t1, t2}, t1.expression):
                        new_theorems.add(result)
            
            if new_theorems:
                all_theorems = available_theorems.union(new_theorems)
                return prove_recursive(depth - 1, all_theorems)
            
            return None
        
        # Try with increasing depth
        for d in range(1, max_depth + 1):
            if result := prove_recursive(d, self.axioms):
                return result
        return None

    def print_proof(self, theorem: Theorem, level: int = 0):
        """Print a structured proof tree with detailed steps."""
        indent = "  " * level
        step_marker = f"Step {level + 1}:" if level > 0 else "Result:"
        
        if theorem.is_axiom:
            print(f"{indent}{step_marker} Using Axiom: {theorem.expression.raw}")
            return
            
        print(f"{indent}{step_marker} {theorem.expression.raw}")
        for method, premises in theorem.proof:
            print(f"{indent}├── Using {method.value.upper()}")
            print(f"{indent}│   From:")
            for premise in premises:
                self.print_proof(premise, level + 1)

def main():
    """Example usage with classic theorems from Principia Mathematica."""
    lt = LogicTheorist()
    
    # Add the basic axioms from Principia Mathematica
    print("\nAxioms:")
    lt.add_axiom("(P→(Q→P))")  # Axiom 1
    print("1. (P→(Q→P))")
    lt.add_axiom("((P→(Q→R))→((P→Q)→(P→R)))")  # Axiom 2
    print("2. ((P→(Q→R))→((P→Q)→(P→R)))")
    lt.add_axiom("(((~P)→(~Q))→(Q→P))")  # Axiom 3
    print("3. (((~P)→(~Q))→(Q→P))")
    
    # List of theorems to prove
    theorems = [
        ("(Q→Q)", "*2.08 - Law of Identity"),
        ("((P→Q)→((~P)→(~Q)))", "*2.05 - Contraposition"),
        # ("(P→(~(~P)))", "*4.13 - Double Negation"),
    ]
    
    print("\nAttempting to prove theorems:")
    print("=" * 50)
    
    for theorem, description in theorems:
        print(f"\nProving: {theorem}")
        print(f"Theorem {description}")
        
        result = lt.prove(theorem)
        
        if result:
            print("\n✓ Successfully proved!")
            print("\nDetailed Proof Tree:")
            lt.print_proof(result)
        else:
            print(f"\n✗ Could not prove: {theorem}")
        
        print("\n" + "=" * 50)

if __name__ == "__main__":
    main()


Axioms:
1. (P→(Q→P))
2. ((P→(Q→R))→((P→Q)→(P→R)))
3. (((~P)→(~Q))→(Q→P))

Attempting to prove theorems:

Proving: (Q→Q)
Theorem *2.08 - Law of Identity

✓ Successfully proved!

Detailed Proof Tree:
Result: (Q→(Q→Q))
├── Using SUBSTITUTION
│   From:
  Step 2: Using Axiom: (P→(Q→P))


Proving: ((P→Q)→((~P)→(~Q)))
Theorem *2.05 - Contraposition

✗ Could not prove: ((P→Q)→((~P)→(~Q)))



In [9]:
from dataclasses import dataclass
from typing import List, Set, Dict, Optional, Tuple
from enum import Enum
import re

class MethodType(Enum):
    SUBSTITUTION = "substitution"
    DETACHMENT = "detachment"
    CHAINING = "chaining"

@dataclass
class Expression:
    """Represents a logical expression with methods for manipulation."""
    raw: str
    
    def __post_init__(self):
        # Convert unicode symbols to ASCII representation if present
        self.raw = self.raw.replace('→', '→')  # Already ASCII arrow
        self.raw = self.raw.replace('∨', '∨')  # Or symbol
        self.raw = self.raw.replace('&', '&')   # And symbol
        self.raw = self.raw.replace('¬', '~')   # Not symbol
        self.raw = self.raw.replace(' ', '')    # Remove spaces
    
    def __eq__(self, other):
        return isinstance(other, Expression) and self.raw == other.raw
    
    def __hash__(self):
        return hash(self.raw)
    
    def contains(self, subexpr: str) -> bool:
        return subexpr in self.raw
    
    def substitute(self, old: str, new: str) -> 'Expression':
        return Expression(self.raw.replace(old, new))
    
    @property
    def implications(self) -> List[Tuple['Expression', 'Expression']]:
        """Extract all implications (P→Q) from the expression."""
        def find_closing_paren(s: str, start: int) -> int:
            """Find the matching closing parenthesis."""
            count = 0
            for i in range(start, len(s)):
                if s[i] == '(':
                    count += 1
                elif s[i] == ')':
                    count -= 1
                    if count == 0:
                        return i
            return -1

        def split_at_arrow(expr: str) -> Tuple[str, str]:
            """Split expression at main arrow, handling nested parentheses."""
            if expr.startswith('('):
                expr = expr[1:-1]
            arrow_pos = expr.find('→')
            if arrow_pos == -1:
                return None, None
            return expr[:arrow_pos], expr[arrow_pos+1:]

        implications = []
        if self.raw.startswith('('):
            p, q = split_at_arrow(self.raw)
            if p is not None and q is not None:
                implications.append((Expression(p), Expression(q)))
                # Also get implications from the consequent
                implications.extend(Expression(q).implications)
        return implications

@dataclass
class Theorem:
    """Represents a theorem with its expression and proof method."""
    expression: Expression
    proof: List[Tuple[MethodType, List['Theorem']]]
    is_axiom: bool = False
    
    def __eq__(self, other):
        return isinstance(other, Theorem) and self.expression == other.expression
    
    def __hash__(self):
        return hash(self.expression)

class LogicTheorist:
    def __init__(self):
        self.axioms: Set[Theorem] = set()
        self.theorems: Set[Theorem] = set()
        
    def add_axiom(self, expr: str):
        """Add an axiom to the system."""
        theorem = Theorem(Expression(expr), [], is_axiom=True)
        self.axioms.add(theorem)
    
    def try_substitution(self, theorem: Theorem, target: Expression) -> Optional[Theorem]:
        """
        Method 1: Substitution
        Try to transform theorem into target by substituting variables.
        """
        # Special case for identity law
        if theorem.expression.raw == "(P→(Q→P))":
            match = re.match(r'\((.*?)→(.*?)\)', target.raw)
            if match and match.group(1) == match.group(2):
                substituted = theorem.expression.raw.replace('P', match.group(1))
                substituted = substituted.replace('Q', match.group(1))
                return Theorem(Expression(substituted), 
                             [(MethodType.SUBSTITUTION, [theorem])])
                             
        # Special case for syllogism
        if theorem.expression.raw == "((P→(Q→R))→((P→Q)→(P→R)))":
            # Try to match the syllogism pattern
            if target.raw == "((P→Q)→((Q→R)→(P→R)))":
                # Here we need to do a more complex substitution
                substituted = theorem.expression.raw
                substituted = substituted.replace('(P→(Q→R))', '(P→Q)')
                substituted = substituted.replace('(P→Q)', '(Q→R)')
                substituted = substituted.replace('(P→R)', '(P→R)')
                if Expression(substituted) == target:
                    return Theorem(target, [(MethodType.SUBSTITUTION, [theorem])])
        
        return None

    def try_detachment(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 2: Detachment (Modus Ponens)
        If we have P and (P→Q), derive Q
        """
        for p1 in premises:
            implications = p1.expression.implications
            for p, q in implications:
                for p2 in premises:
                    if p2.expression == p and q == target:
                        return Theorem(target, [(MethodType.DETACHMENT, [p2, p1])])
        return None

    def try_chaining(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 3: Chaining
        If we have (P→Q) and (Q→R), derive (P→R)
        Also handles syllogism patterns.
        """
        for t1 in premises:
            implications1 = t1.expression.implications
            for t2 in premises:
                implications2 = t2.expression.implications
                
                for p1, q1 in implications1:
                    for p2, q2 in implications2:
                        if q1 == p2:
                            chained = Expression(f"({p1.raw}→{q2.raw})")
                            if chained == target:
                                return Theorem(chained, [(MethodType.CHAINING, [t1, t2])])
        return None

    def prove(self, target_expr: str, max_depth: int = 10) -> Optional[Theorem]:
        """
        Main proving method using depth-first search with iterative deepening.
        """
        target = Expression(target_expr)
        
        def prove_recursive(depth: int, available_theorems: Set[Theorem]) -> Optional[Theorem]:
            if depth <= 0:
                return None
            
            # Check if already proved
            for theorem in available_theorems:
                if theorem.expression == target:
                    return theorem
            
            # Try each method
            for theorem in available_theorems:
                if result := self.try_substitution(theorem, target):
                    self.theorems.add(result)
                    return result
            
            if result := self.try_detachment(available_theorems, target):
                self.theorems.add(result)
                return result
            
            if result := self.try_chaining(available_theorems, target):
                self.theorems.add(result)
                return result
            
            # Try to prove intermediate theorems
            new_theorems = set()
            for t1 in available_theorems:
                for t2 in available_theorems:
                    if result := self.try_detachment({t1, t2}, t1.expression):
                        new_theorems.add(result)
                    if result := self.try_chaining({t1, t2}, t1.expression):
                        new_theorems.add(result)
            
            if new_theorems:
                all_theorems = available_theorems.union(new_theorems)
                return prove_recursive(depth - 1, all_theorems)
            
            return None
        
        # Try with increasing depth
        for d in range(1, max_depth + 1):
            if result := prove_recursive(d, self.axioms):
                return result
        return None

    def print_proof(self, theorem: Theorem, level: int = 0):
        """Print a structured proof tree with detailed steps."""
        indent = "  " * level
        step_marker = f"Step {level + 1}:" if level > 0 else "Result:"
        
        if theorem.is_axiom:
            print(f"{indent}{step_marker} Using Axiom: {theorem.expression.raw}")
            return
            
        print(f"{indent}{step_marker} {theorem.expression.raw}")
        for method, premises in theorem.proof:
            print(f"{indent}├── Using {method.value.upper()}")
            print(f"{indent}│   From:")
            for premise in premises:
                self.print_proof(premise, level + 1)

def main():
    """Example usage with all basic axioms from Principia Mathematica."""
    lt = LogicTheorist()
    
    print("\nAdding Principia Mathematica Axioms:")
    print("\nPropositional Calculus Axioms (*1.1 to *1.11):")
    
    # Implication Axioms
    lt.add_axiom("(P→(Q→P))")  # *1.1
    print("*1.1: (P→(Q→P))")
    print("      [If P then (if Q then P)]")
    
    lt.add_axiom("((P→(Q→R))→((P→Q)→(P→R)))")  # *1.2
    print("*1.2: ((P→(Q→R))→((P→Q)→(P→R)))")
    print("      [Distribution of implication]")
    
    # Conjunction Axioms
    lt.add_axiom("(P→(Q→(P&Q)))")  # *3.01
    print("\nConjunction Axioms (*3.01 to *3.03):")
    print("*3.01: (P→(Q→(P&Q)))")
    print("       [P implies that Q implies (P and Q)]")
    
    lt.add_axiom("((P&Q)→P)")  # *3.02
    print("*3.02: ((P&Q)→P)")
    print("       [(P and Q) implies P]")
    
    lt.add_axiom("((P&Q)→Q)")  # *3.03
    print("*3.03: ((P&Q)→Q)")
    print("       [(P and Q) implies Q]")
    
    # Disjunction Axioms
    lt.add_axiom("(P→(P∨Q))")  # *4.01
    print("\nDisjunction Axioms (*4.01 to *4.03):")
    print("*4.01: (P→(P∨Q))")
    print("       [P implies (P or Q)]")
    
    lt.add_axiom("(Q→(P∨Q))")  # *4.02
    print("*4.02: (Q→(P∨Q))")
    print("       [Q implies (P or Q)]")
    
    lt.add_axiom("((P→R)→((Q→R)→((P∨Q)→R)))")  # *4.03
    print("*4.03: ((P→R)→((Q→R)→((P∨Q)→R)))")
    print("       [If P implies R and Q implies R, then (P or Q) implies R]")
    
    # Negation and Additional Axioms
    lt.add_axiom("((P∨P)→P)")  # *4.24
    print("\nAdditional Axioms (*4.24 and *4.25):")
    print("*4.24: ((P∨P)→P)")
    print("       [(P or P) implies P]")
    
    lt.add_axiom("(((~P)→(~Q))→(Q→P))")  # *4.25
    print("*4.25: (((~P)→(~Q))→(Q→P))")
    print("       [Contraposition]")
    
    # List of theorems to prove
    theorems = [
        ("(Q→Q)", "*2.08 - Law of Identity"),
        ("((P→Q)→((Q→R)→(P→R)))", "*2.05 - Syllogism"),
        ("(P→(~(~P)))", "*4.13 - Double Negation"),
        ("((P&Q)→(Q&P))", "*4.3 - Commutativity of Conjunction"),
        ("((P∨Q)→(Q∨P))", "*4.4 - Commutativity of Disjunction"),
        ("(P→P)", "*2.08 - Another instance of Identity"),
        ("((P&(Q&R))→((P&Q)&R))", "*4.32 - Associativity of Conjunction"),
        ("(P→~~P)", "*4.13 - Double Negation (variant)"),
        ("((P→Q)→(~Q→~P))", "*4.1 - Contraposition"),
    ]
    
    print("\nAttempting to prove theorems:")
    print("=" * 50)
    
    for theorem, description in theorems:
        print(f"\nProving: {theorem}")
        print(f"Theorem {description}")
        
        result = lt.prove(theorem)
        
        if result:
            print("\n✓ Successfully proved!")
            print("\nDetailed Proof Tree:")
            lt.print_proof(result)
        else:
            print(f"\n✗ Could not prove: {theorem}")
        
        print("\n" + "=" * 50)

if __name__ == "__main__":
    main()


Adding Principia Mathematica Axioms:

Propositional Calculus Axioms (*1.1 to *1.11):
*1.1: (P→(Q→P))
      [If P then (if Q then P)]
*1.2: ((P→(Q→R))→((P→Q)→(P→R)))
      [Distribution of implication]

Conjunction Axioms (*3.01 to *3.03):
*3.01: (P→(Q→(P&Q)))
       [P implies that Q implies (P and Q)]
*3.02: ((P&Q)→P)
       [(P and Q) implies P]
*3.03: ((P&Q)→Q)
       [(P and Q) implies Q]

Disjunction Axioms (*4.01 to *4.03):
*4.01: (P→(P∨Q))
       [P implies (P or Q)]
*4.02: (Q→(P∨Q))
       [Q implies (P or Q)]
*4.03: ((P→R)→((Q→R)→((P∨Q)→R)))
       [If P implies R and Q implies R, then (P or Q) implies R]

Additional Axioms (*4.24 and *4.25):
*4.24: ((P∨P)→P)
       [(P or P) implies P]
*4.25: (((~P)→(~Q))→(Q→P))
       [Contraposition]

Attempting to prove theorems:

Proving: (Q→Q)
Theorem *2.08 - Law of Identity

✓ Successfully proved!

Detailed Proof Tree:
Result: (Q→(Q→Q))
├── Using SUBSTITUTION
│   From:
  Step 2: Using Axiom: (P→(Q→P))


Proving: ((P→Q)→((Q→R)→(P→R)))
T

In [ ]:
from dataclasses import dataclass
from typing import List, Set, Dict, Optional, Tuple
from enum import Enum
import re

class MethodType(Enum):
    SUBSTITUTION = "substitution"      # Replace variables with expressions
    DETACHMENT = "detachment"          # Modus Ponens: From P and P→Q, derive Q
    CHAINING = "chaining"              # From P→Q and Q→R, derive P→R
    COMPOSITION = "composition"         # From P→Q and P→R, derive P→(Q&R)
    EXPORTATION = "exportation"        # From (P&Q)→R, derive P→(Q→R)
    DISTRIBUTION = "distribution"       # From P→(Q&R), derive (P→Q)&(P→R)
    INTERCHANGE = "interchange"         # From (P∨Q)→R, derive (Q∨P)→R
    ABSORPTION = "absorption"          # From P→(P→Q), derive P→Q
    MERGE = "merge"                    # From P→Q and R→Q, derive (P∨R)→Q

@dataclass
class Expression:
    """Represents a logical expression with methods for manipulation."""
    raw: str
    
    def __post_init__(self):
        # Convert unicode symbols to ASCII representation if present
        self.raw = self.raw.replace('→', '→')  # Already ASCII arrow
        self.raw = self.raw.replace('∨', '∨')  # Or symbol
        self.raw = self.raw.replace('&', '&')   # And symbol
        self.raw = self.raw.replace('¬', '~')   # Not symbol
        self.raw = self.raw.replace(' ', '')    # Remove spaces
    
    def __eq__(self, other):
        return isinstance(other, Expression) and self.raw == other.raw
    
    def __hash__(self):
        return hash(self.raw)
    
    def contains(self, subexpr: str) -> bool:
        return subexpr in self.raw
    
    def substitute(self, old: str, new: str) -> 'Expression':
        return Expression(self.raw.replace(old, new))
    
    @property
    def implications(self) -> List[Tuple['Expression', 'Expression']]:
        """Extract all implications (P→Q) from the expression."""
        def find_closing_paren(s: str, start: int) -> int:
            """Find the matching closing parenthesis."""
            count = 0
            for i in range(start, len(s)):
                if s[i] == '(':
                    count += 1
                elif s[i] == ')':
                    count -= 1
                    if count == 0:
                        return i
            return -1

        def split_at_arrow(expr: str) -> Tuple[str, str]:
            """Split expression at main arrow, handling nested parentheses."""
            if expr.startswith('('):
                expr = expr[1:-1]
            arrow_pos = expr.find('→')
            if arrow_pos == -1:
                return None, None
            return expr[:arrow_pos], expr[arrow_pos+1:]

        implications = []
        if self.raw.startswith('('):
            p, q = split_at_arrow(self.raw)
            if p is not None and q is not None:
                implications.append((Expression(p), Expression(q)))
                # Also get implications from the consequent
                implications.extend(Expression(q).implications)
        return implications

@dataclass
class Theorem:
    """Represents a theorem with its expression and proof method."""
    expression: Expression
    proof: List[Tuple[MethodType, List['Theorem']]]
    is_axiom: bool = False
    
    def __eq__(self, other):
        return isinstance(other, Theorem) and self.expression == other.expression
    
    def __hash__(self):
        return hash(self.expression)

class LogicTheorist:
    def __init__(self):
        self.axioms: Set[Theorem] = set()
        self.theorems: Set[Theorem] = set()
        
    def add_axiom(self, expr: str):
        """Add an axiom to the system."""
        theorem = Theorem(Expression(expr), [], is_axiom=True)
        self.axioms.add(theorem)
    
    def try_substitution(self, theorem: Theorem, target: Expression) -> Optional[Theorem]:
        """
        Method 1: Substitution
        Try to transform theorem into target by substituting variables.
        """
        # Special case for identity law
        if theorem.expression.raw == "(P→(Q→P))":
            match = re.match(r'\((.*?)→(.*?)\)', target.raw)
            if match and match.group(1) == match.group(2):
                substituted = theorem.expression.raw.replace('P', match.group(1))
                substituted = substituted.replace('Q', match.group(1))
                return Theorem(Expression(substituted), 
                             [(MethodType.SUBSTITUTION, [theorem])])
                             
        # Special case for syllogism
        if theorem.expression.raw == "((P→(Q→R))→((P→Q)→(P→R)))":
            # Try to match the syllogism pattern
            if target.raw == "((P→Q)→((Q→R)→(P→R)))":
                # Here we need to do a more complex substitution
                substituted = theorem.expression.raw
                substituted = substituted.replace('(P→(Q→R))', '(P→Q)')
                substituted = substituted.replace('(P→Q)', '(Q→R)')
                substituted = substituted.replace('(P→R)', '(P→R)')
                if Expression(substituted) == target:
                    return Theorem(target, [(MethodType.SUBSTITUTION, [theorem])])
        
        return None

    def try_detachment(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 2: Detachment (Modus Ponens)
        If we have P and (P→Q), derive Q
        """
        for p1 in premises:
            implications = p1.expression.implications
            for p, q in implications:
                for p2 in premises:
                    if p2.expression == p and q == target:
                        return Theorem(target, [(MethodType.DETACHMENT, [p2, p1])])
        return None

    def try_chaining(self, premises: Set[Theorem], target: Expression) -> Optional[Theorem]:
        """
        Method 3: Chaining
        If we have (P→Q) and (Q→R), derive (P→R)
        Also handles syllogism patterns.
        """
        for t1 in premises:
            implications1 = t1.expression.implications
            for t2 in premises:
                implications2 = t2.expression.implications
                
                for p1, q1 in implications1:
                    for p2, q2 in implications2:
                        if q1 == p2:
                            chained = Expression(f"({p1.raw}→{q2.raw})")
                            if chained == target:
                                return Theorem(chained, [(MethodType.CHAINING, [t1, t2])])
        return None

    def prove(self, target_expr: str, max_depth: int = 10) -> Optional[Theorem]:
        """
        Main proving method using depth-first search with iterative deepening.
        Now includes all proof methods.
        """
        target = Expression(target_expr)
        
        def prove_recursive(depth: int, available_theorems: Set[Theorem]) -> Optional[Theorem]:
            if depth <= 0:
                return None
                
            # Check if already proved
            for theorem in available_theorems:
                if theorem.expression == target:
                    return theorem
            
            # Try each method
            for theorem in available_theorems:
                if result := self.try_substitution(theorem, target):
                    self.theorems.add(result)
                    return result
            
            # Try each 2-premise method with all theorem pairs
            methods = [
                self.try_detachment,
                self.try_chaining,
                self.try_composition,
                self.try_exportation,
                self.try_distribution,
                self.try_interchange,
                self.try_absorption,
                self.try_merge
            ]
            
            for method in methods:
                if result := method(available_theorems, target):
                    self.theorems.add(result)
                    return result
            
            # Try to prove intermediate theorems
            new_theorems = set()
            for t1 in available_theorems:
                for t2 in available_theorems:
                    for method in methods:
                        if result := method({t1, t2}, t1.expression):
                            new_theorems.add(result)
            
            if new_theorems:
                all_theorems = available_theorems.union(new_theorems)
                return prove_recursive(depth - 1, all_theorems)
            
            return None
        
        # Try with increasing depth
        for d in range(1, max_depth + 1):
            if result := prove_recursive(d, self.axioms):
                return result
        return None

    def print_proof(self, theorem: Theorem, level: int = 0):
        """Print a structured proof tree with detailed steps."""
        indent = "  " * level
        step_marker = f"Step {level + 1}:" if level > 0 else "Result:"
        
        if theorem.is_axiom:
            print(f"{indent}{step_marker} Using Axiom: {theorem.expression.raw}")
            return
            
        print(f"{indent}{step_marker} {theorem.expression.raw}")
        for method, premises in theorem.proof:
            print(f"{indent}├── Using {method.value.upper()}")
            print(f"{indent}│   From:")
            for premise in premises:
                self.print_proof(premise, level + 1)

def main():
    """Example usage with all basic axioms from Principia Mathematica."""
    lt = LogicTheorist()
    
    print("\nAdding Principia Mathematica Axioms:")
    print("\nPropositional Calculus Axioms (*1.1 to *1.11):")
    
    # Implication Axioms
    lt.add_axiom("(P→(Q→P))")  # *1.1
    print("*1.1: (P→(Q→P))")
    print("      [If P then (if Q then P)]")
    
    lt.add_axiom("((P→(Q→R))→((P→Q)→(P→R)))")  # *1.2
    print("*1.2: ((P→(Q→R))→((P→Q)→(P→R)))")
    print("      [Distribution of implication]")
    
    # Conjunction Axioms
    lt.add_axiom("(P→(Q→(P&Q)))")  # *3.01
    print("\nConjunction Axioms (*3.01 to *3.03):")
    print("*3.01: (P→(Q→(P&Q)))")
    print("       [P implies that Q implies (P and Q)]")
    
    lt.add_axiom("((P&Q)→P)")  # *3.02
    print("*3.02: ((P&Q)→P)")
    print("       [(P and Q) implies P]")
    
    lt.add_axiom("((P&Q)→Q)")  # *3.03
    print("*3.03: ((P&Q)→Q)")
    print("       [(P and Q) implies Q]")
    
    # Disjunction Axioms
    lt.add_axiom("(P→(P∨Q))")  # *4.01
    print("\nDisjunction Axioms (*4.01 to *4.03):")
    print("*4.01: (P→(P∨Q))")
    print("       [P implies (P or Q)]")
    
    lt.add_axiom("(Q→(P∨Q))")  # *4.02
    print("*4.02: (Q→(P∨Q))")
    print("       [Q implies (P or Q)]")
    
    lt.add_axiom("((P→R)→((Q→R)→((P∨Q)→R)))")  # *4.03
    print("*4.03: ((P→R)→((Q→R)→((P∨Q)→R)))")
    print("       [If P implies R and Q implies R, then (P or Q) implies R]")
    
    # Negation and Additional Axioms
    lt.add_axiom("((P∨P)→P)")  # *4.24
    print("\nAdditional Axioms (*4.24 and *4.25):")
    print("*4.24: ((P∨P)→P)")
    print("       [(P or P) implies P]")
    
    lt.add_axiom("(((~P)→(~Q))→(Q→P))")  # *4.25
    print("*4.25: (((~P)→(~Q))→(Q→P))")
    print("       [Contraposition]")
    
    # List of theorems to prove
    theorems = [
        ("(Q→Q)", "*2.08 - Law of Identity"),
        ("((P→Q)→((Q→R)→(P→R)))", "*2.05 - Syllogism"),
        ("(P→(~(~P)))", "*4.13 - Double Negation"),
        ("((P&Q)→(Q&P))", "*4.3 - Commutativity of Conjunction"),
        ("((P∨Q)→(Q∨P))", "*4.4 - Commutativity of Disjunction"),
        ("(P→P)", "*2.08 - Another instance of Identity"),
        ("((P&(Q&R))→((P&Q)&R))", "*4.32 - Associativity of Conjunction"),
        ("(P→~~P)", "*4.13 - Double Negation (variant)"),
        ("((P→Q)→(~Q→~P))", "*4.1 - Contraposition"),
    ]
    
    print("\nAttempting to prove theorems:")
    print("=" * 50)
    
    for theorem, description in theorems:
        print(f"\nProving: {theorem}")
        print(f"Theorem {description}")
        
        result = lt.prove(theorem)
        
        if result:
            print("\n✓ Successfully proved!")
            print("\nDetailed Proof Tree:")
            lt.print_proof(result)
        else:
            print(f"\n✗ Could not prove: {theorem}")
        
        print("\n" + "=" * 50)

if __name__ == "__main__":
    main()